In [2]:
import pandas as pd
import numpy as np
from cvxpy import *

In [3]:
# read monthly_prices.csv
mp = pd.read_csv("monthly_prices.csv",index_col=0)
mr = pd.DataFrame()
mp.head()

,MSFT,V,WMT
1,44.259998,69.660004,64.839996
2,52.639999,77.580002,57.240002
3,54.349998,79.010002,58.840000
4,55.480000,77.550003,61.299999
5,55.090000,74.489998,66.360001


In [4]:
# compute monthly returns
for s in mp.columns:
    date = mp.index[0]
    pr0 = mp[s][date] 
    for t in range(1,len(mp.index)):
        date = mp.index[t]
        pr1 = mp[s][date]
        ret = (pr1-pr0)/pr0
        #mr.set_value(date,s,ret)
        mr.at[date,s]=ret
        pr0 = pr1
        
# get symbol names
symbols = mr.columns

In [5]:
# convert monthly return data frame to a numpy matrix
#return_data = mr.as_matrix().T
return_data = mr.values.T

In [6]:
# compute mean return
r = np.asarray(np.mean(return_data, axis=1))

# covariance
C = np.asmatrix(np.cov(return_data))

# print out expected return and std deviation
print("----------------------")
for j in range(len(symbols)):
    print('%s: Exp ret = %f, Risk = %f' %(symbols[j],r[j], C[j,j]**0.5))

----------------------
MSFT: Exp ret = 0.024611, Risk = 0.058040
V: Exp ret = 0.018237, Risk = 0.042807
WMT: Exp ret = 0.009066, Risk = 0.044461


In [18]:
# set up optimization model
n = len(symbols)
x = Variable(n)
req_return = 0.02
ret = r.T@x
risk = quad_form(x, C)
prob = Problem(Minimize(risk), 
               [sum(x) == 1, ret >= req_return, x >= 0])


In [19]:
# solve problem and write solution
try:
    prob.solve()
    print("----------------------")
    print("Optimal portfolio")
    print("----------------------")
    for s in range(len(symbols)):
        #print('x[%s] = %f'%(symbols[s],x.value[s,0]))
        print('x[%s] = %f'%(symbols[s],x.value[s]))
    print("----------------------")
    print('Exp ret = %f' %(ret.value))
    print('risk    = %f' %((risk.value)**0.5))
    print("----------------------")
except:
    print('Error')

----------------------
Optimal portfolio
----------------------
x[MSFT] = 0.582818
x[V] = 0.204324
x[WMT] = 0.212858
----------------------
Exp ret = 0.020000
risk    = 0.038256
----------------------


In [20]:
symbols

Index(['MSFT', 'V', 'WMT'], dtype='object')

In [21]:
x[0]

Expression(AFFINE, UNKNOWN, ())

In [22]:
x.value[0]

0.5828175485022873